In [ ]:
import pandas as pd
from IPython.display import display

GEM_df = pd.read_excel("data/EU_GEM_data.xlsx")
WDI_df = pd.read_excel("data/EU_WDI_data.xlsx")

GEM_df = GEM_df.drop(columns=['Time Code', 'CPI Price, % y-o-y, median weighted, seas. adj., [CPTOTSAXMZGY]', 'Nominal Effective Exchange Rate,,,, [NEER]', 'Real Effective Exchange Rate,,,, [REER]'])
WDI_df = WDI_df.drop(columns=['Time Code', 'Country Name', "Educational attainment, at least Master's or equivalent, population 25+, total (%) (cumulative) [SE.TER.CUAT.MS.ZS]", "Educational attainment, Doctoral or equivalent, population 25+, total (%) (cumulative) [SE.TER.CUAT.DO.ZS]"])
WDI_df = WDI_df.rename({"Time": "Year"}, axis=1)

GEM_df = GEM_df.reset_index(drop=True)
WDI_df = WDI_df.reset_index(drop=True)

GEM_df['Year'] = GEM_df['Time'].str[:4]
GEM_df['Time'] = GEM_df['Time'].astype(str)
GEM_df["Year"] = pd.to_numeric(GEM_df["Year"], errors='coerce')
GEM_df['Country Code'] = GEM_df['Country Code'].astype(str)
WDI_df["Year"] = pd.to_numeric(WDI_df["Year"], errors='coerce')
WDI_df['Country Code'] = WDI_df['Country Code'].astype(str)

df = pd.merge(GEM_df, WDI_df, on=["Year", "Country Code"], how="left")

# print(GEM_df[['Year', 'Country Code']].dtypes)
# print(WDI_df.reset_index()[['Year', 'Country Code']].dtypes)

# print("GEM_df index:", GEM_df.index)
# print("GEM_df columns:", GEM_df.columns)

# print("WDI_df index:", WDI_df.index)
# print("WDI_df columns:", WDI_df.columns)

count_countries = df["Country Code"].value_counts()
print(len(count_countries))
display(df.head(3))
# df.to_excel("data/EU_Joined_data.xlsx")

34


,Time,Country,Country Code,"CPI Price, % y-o-y, not seas. adj.,, [CPTOTSAXNZGY]","Exports Merchandise, Customs, current US$, millions, not seas. adj. [DXGSRMRCHNSCD]","Exports Merchandise, Customs, current US$, millions, seas. adj. [DXGSRMRCHSACD]","Imports Merchandise, Customs, current US$, millions, not seas. adj. [DMGSRMRCHNSCD]","Imports Merchandise, Customs, current US$, millions, seas. adj. [DMGSRMRCHSACD]","Industrial Production, constant US$, seas. adj.,, [IPTOTSAKD]","Industrial Production, constant US$,,, [IPTOTNSKD]",...,Labor force with basic education (% of total working-age population with basic education) [SL.TLF.BASC.ZS],Labor force with intermediate education (% of total working-age population with intermediate education) [SL.TLF.INTM.ZS],"Educational attainment, at least Bachelor's or equivalent, population 25+, total (%) (cumulative) [SE.TER.CUAT.BA.ZS]","Educational attainment, at least completed lower secondary, population 25+, total (%) (cumulative) [SE.SEC.CUAT.LO.ZS]","Educational attainment, at least completed post-secondary, population 25+, total (%) (cumulative) [SE.SEC.CUAT.PO.ZS]","Educational attainment, at least completed primary, population 25+ years, total (%) (cumulative) [SE.PRM.CUAT.ZS]","Educational attainment, at least completed short-cycle tertiary, population 25+, total (%) (cumulative) [SE.TER.CUAT.ST.ZS]","Educational attainment, at least completed upper secondary, population 25+, total (%) (cumulative) [SE.SEC.CUAT.UP.ZS]","Educational attainment, at least Master's or equivalent, population 25+, total (%) (cumulative) [SE.TER.CUAT.MS.ZS]","Educational attainment, Doctoral or equivalent, population 25+, total (%) (cumulative) [SE.TER.CUAT.DO.ZS]"
0,1987M01,Austria,AUT,0.6,..,..,..,..,..,..,...,..,..,..,..,..,..,..,..,..,..
1,1987M01,Belgium,BEL,0.906828,..,..,..,..,..,..,...,..,..,..,..,..,..,..,..,..,..
2,1987M01,Croatia,HRV,91.792625,..,..,..,..,..,..,...,..,..,..,..,..,..,..,..,..,..


In [ ]:
joined_df = df

for col in joined_df.columns:
    if col not in ["Time", "Country Code", "Country"]:
        joined_df[col] = pd.to_numeric(joined_df[col], errors='coerce')

joined_df['Time'] = joined_df['Time'].astype("string")
joined_df['Country Code'] = joined_df['Country Code'].astype("string")
joined_df['Country'] = joined_df['Country'].astype("string")


joined_df = joined_df[~joined_df['Country Code'].isin(['EST', 'MLT', 'LTU', 'HRV', 'CYP', 'CZE', 'GRC', 'SVK', 'AUT', 'FRA', 'HUN', 'YUG', 'CHE', 'GEO', 'RUS', 'BGR', 'SVN', 'DNK'])]
joined_df['Year'] = pd.to_numeric(joined_df['Year'], errors='coerce')
joined_df = joined_df[joined_df['Year'] > 2009]
joined_df = joined_df[joined_df['Time'] < "2024M04"]
joined_df = joined_df.sort_values(by=["Country Code", "Time"])



print(joined_df.isna().sum().sum())

# count_countries = joined_df["Country Code"].value_counts()
# print(len(count_countries))

for col in joined_df.columns:
    joined_df[col] = joined_df.groupby(["Country Code"])[col].ffill()

joined_df = joined_df.reset_index()
print(joined_df.dtypes)
joined_df.to_excel("data/EU_cleaned_data.xlsx", index=False)

print(joined_df.isna().sum().sum())
print(joined_df.shape)
print(joined_df.groupby("Country Code").apply(lambda x: x.isnull().sum().sum()))
display(joined_df.head(3))



/tmp/ipykernel_982/1494354638.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  joined_df['Year'] = pd.to_numeric(joined_df['Year'], errors='coerce')


2913
index                                                                                                                                   int64
Time                                                                                                                           string[python]
Country                                                                                                                        string[python]
Country Code                                                                                                                   string[python]
CPI Price, % y-o-y, not seas. adj.,, [CPTOTSAXNZGY]                                                                                   float64
Exports Merchandise, Customs, current US$, millions, not seas. adj. [DXGSRMRCHNSCD]                                                   float64
Exports Merchandise, Customs, current US$, millions, seas. adj. [DXGSRMRCHSACD]                                                       float64
I

/tmp/ipykernel_982/1494354638.py:34: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  print(joined_df.groupby("Country Code").apply(lambda x: x.isnull().sum().sum()))


,index,Time,Country,Country Code,"CPI Price, % y-o-y, not seas. adj.,, [CPTOTSAXNZGY]","Exports Merchandise, Customs, current US$, millions, not seas. adj. [DXGSRMRCHNSCD]","Exports Merchandise, Customs, current US$, millions, seas. adj. [DXGSRMRCHSACD]","Imports Merchandise, Customs, current US$, millions, not seas. adj. [DMGSRMRCHNSCD]","Imports Merchandise, Customs, current US$, millions, seas. adj. [DMGSRMRCHSACD]","Industrial Production, constant US$, seas. adj.,, [IPTOTSAKD]",...,Labor force with basic education (% of total working-age population with basic education) [SL.TLF.BASC.ZS],Labor force with intermediate education (% of total working-age population with intermediate education) [SL.TLF.INTM.ZS],"Educational attainment, at least Bachelor's or equivalent, population 25+, total (%) (cumulative) [SE.TER.CUAT.BA.ZS]","Educational attainment, at least completed lower secondary, population 25+, total (%) (cumulative) [SE.SEC.CUAT.LO.ZS]","Educational attainment, at least completed post-secondary, population 25+, total (%) (cumulative) [SE.SEC.CUAT.PO.ZS]","Educational attainment, at least completed primary, population 25+ years, total (%) (cumulative) [SE.PRM.CUAT.ZS]","Educational attainment, at least completed short-cycle tertiary, population 25+, total (%) (cumulative) [SE.TER.CUAT.ST.ZS]","Educational attainment, at least completed upper secondary, population 25+, total (%) (cumulative) [SE.SEC.CUAT.UP.ZS]","Educational attainment, at least Master's or equivalent, population 25+, total (%) (cumulative) [SE.TER.CUAT.MS.ZS]","Educational attainment, Doctoral or equivalent, population 25+, total (%) (cumulative) [SE.TER.CUAT.DO.ZS]"
0,9109,2010M01,Belgium,BEL,0.559333,31749.7,33948.521959,30738.367137,31939.022977,8.290634e+09,...,32.989,62.954,0.541977,79.639999,31.74,92.660004,30.0,61.73,0.0,0.0
1,9142,2010M02,Belgium,BEL,0.646717,31937.3,33085.117019,30109.538106,31632.736466,8.412735e+09,...,32.989,62.954,0.541977,79.639999,31.74,92.660004,30.0,61.73,0.0,0.0
2,9175,2010M03,Belgium,BEL,1.599559,38176.9,34288.929721,35651.256763,32272.402075,8.388315e+09,...,32.989,62.954,0.541977,79.639999,31.74,92.660004,30.0,61.73,0.0,0.0
